# DBT Tests

### Introduction

Another benefit of DBT is that it allows us to test that our data.  For example, we can use tests to ensure that no data in a specific column has null values, or that all values in a column are unique.  In this lesson, we'll see how to DBT can allow to write some tests to ensure data quality.

### Schema Testing Options

With DBT, we can use tests that DBT provides for us out of the box.  Let's start.  

The following helpers that DBT provides to test our data are the following:

* `unique`: Assert that all values in a column are unique

* `not_null`: Assert that no values in a column are null

* `accepted_values`: Assert that all values in a column are one of the accepted values

* `relationships`: Ensure that every foreign key maps to a primary key in the other model.

### Adding a test

Ok, so now that we see what we can do with DBT, let's begin to add our first test.  We'll start by adding tests for some of the models in our `staging` folder.

Under our hubspot folder, we can add a new file called `schema.yml` and add the following to that file.

```yaml
version: 2

models: 
  - name: stg_hubspot_contacts
    columns:
      - name: first_name
        tests:
          - not_null
      - name: last_name
        tests:
          - not_null
```

The yaml above asserts that in the `stg_hubspot_contacts` model, the `first_name` and `last_name` columns do not have any null values.  Now let's move through the details.

At the top, the `version: 2` indicates the version of the DBT api that we are using.  Then, because we can assert tests for multiple `models` within the same test file, we then added the `models` key.  Then, we moved in two spaces and specified the name of the columns we wanted to test in the model.  And then we added a key for tests, and specified each test.

> So with yaml, we indicate that one entity is a subset of another by moving in two spaces.  And, as for those dashes, we need a dash before each element of a list.

### Running the tests

Now it's time to again run our tests against the data.  

In our DBT command line at the bottom, we can run tests by typing in `dbt test` and then pressing return. 

> Or, if we want to only run tests of a specific model, we can do so with a call to `dbt test --models stg_hubspot_contacts`.

Upon doing so, we should see something like the following:

> <img src="./dbt_test.png" width="100%">

In the dashboard above, we can view both the `logs` of the entire run, and there is also separate entry for our specific test: `not_null_stg_hubspot_contacts_first_name`.  If we click on that `not_null_stg_hubspot_contacts_first_name` drop down, we see more information.

> <img src="./first_test.png" width="80%">

Above, it just tells us that the test passed -- not too useful.  But if we then click on `details`, we can also see the specific SQL that was run.

> <img src="./generated_cte.png" width="100%">

So above, the key part is in the second select statement -- our subquery.  In the subquery, DBT is selecting from the `stg_hubspot_contacts` table for any places where the first `first_name` is null.  Then, in the first select, when the subquery returns any records, it will run an error and a warning.

### Debugging Tests

Now let's move through the procedure we can perform when a test does fail.  Let's say that we want to assert the following:

* All first names must be either `fran` or `daphne`

We can add that test by updating our yaml to be the following:

```yaml
version: 2

models: 
  - name: stg_hubspot_contacts
    columns:
      - name: first_name
        tests:
          - not_null
          - accepted_values:
              values: ['fran', 'daphne']
      - name: last_name
        tests:
          - not_null
```

Ok, now let's run the tests.

> <img src="./debugging-tests.png" width="100%">

Ok, so we can see that this time we can see that something is broken.  And if we want to identify what data is broken, then we can click on the details of the test.

<img src="./query-test.png" width="60%">

We can then copy that query into snowflake.

<img src="./snowflake-query.png" width="100%">

### Resources

[DBT Test Documentation](https://docs.getdbt.com/reference/resource-properties/tests)